In [1]:
# Load the GEO Data 

In [2]:
import GEOparse
import pandas as pd
import sklearn.cluster

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
gse = GEOparse.get_GEO(filepath="./GSE10245_family.soft.gz")

06-Dec-2022 17:36:13 INFO GEOparse - Parsing ./GSE10245_family.soft.gz: 
06-Dec-2022 17:36:13 DEBUG GEOparse - DATABASE: GeoMiame
06-Dec-2022 17:36:13 DEBUG GEOparse - SERIES: GSE10245
06-Dec-2022 17:36:13 DEBUG GEOparse - PLATFORM: GPL570
/opt/anaconda3/lib/python3.9/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
06-Dec-2022 17:36:14 DEBUG GEOparse - SAMPLE: GSM258551
06-Dec-2022 17:36:14 DEBUG GEOparse - SAMPLE: GSM258552
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258553
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258554
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258555
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258556
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258557
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258558
06-Dec-2022 17:36:15 DEBUG GEOparse - SAMPLE: GSM258559
06-Dec-2022 17:36:15 DEBUG 

In [5]:
matrix_data = gse.pivot_samples("VALUE").T

In [6]:
matrix_data.head()

ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,AFFX-r2-Bs-thr-M_s_at,AFFX-r2-Ec-bioB-3_at,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioB-M_at,AFFX-r2-Ec-bioC-3_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at
name,,,,,,,,,,,,,,,,,,,,,
GSM258551,9.129905,8.034022,3.564520,4.746490,2.320698,5.519153,3.339182,2.775395,8.303437,2.981327,...,10.297589,8.857847,9.369037,9.435077,10.936404,10.730983,13.599488,13.031726,15.028729,14.586347
GSM258552,9.843349,7.973332,4.994852,5.197306,2.248520,5.081258,2.934516,2.617097,9.145519,4.619668,...,8.217455,7.865127,8.116953,8.306683,9.986505,9.719266,12.847711,12.250033,14.440756,14.072366
GSM258553,9.730661,8.834045,5.066018,5.234618,2.259504,4.657257,3.007192,2.634559,9.159054,2.746464,...,8.713199,6.996104,7.803869,8.029131,9.468096,9.101115,12.384142,11.798363,14.439887,14.011392
GSM258554,9.032165,7.723965,4.958580,6.078180,2.262787,4.535683,3.167112,3.127495,7.770354,6.212399,...,8.805233,8.221659,8.434090,8.609512,10.064087,9.763076,12.969199,12.307684,14.557363,14.162145
GSM258555,10.281793,9.040800,4.951835,5.205632,2.207531,3.731919,2.711790,2.707079,6.380928,4.528499,...,8.285092,8.042156,8.174685,8.620098,10.005558,9.651785,12.924465,12.243207,14.612223,14.122751


In [7]:
charac_list = []
for gsm in gse.gsms.values():
    charac_list.append(gsm.metadata["characteristics_ch1"][0].split(":")[1][1:])

In [8]:
charac_list

['adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'squamous cell carcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'adenocarcinoma',
 'squamous cell carcinoma',
 'adenocarcinoma',
 'squamous

In [11]:
# Below are part 4,5,6

In [9]:
# Accuracy of training data with model trained on training data 
def compute_all_accuracy(data,subtype,seed): 
    #develope the kmeans model
    kmeans = KMeans(n_clusters = 2, random_state = seed).fit(data)
    
    # Predict label(in the form of array)
    predict_label = kmeans.labels_
    # convert predict label array into list 
    predict_label = predict_label.tolist()
    
    # Create a dataframe for predict label and subtype
    df = pd.DataFrame([predict_label,subtype]).T
    df.columns = ["Cluster", "Subtype"]
    
    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]
    
    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]
    
    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}
    
    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in subtype]
    # Compute accuracy_score 
    return accuracy_score(train_label, predict_label)

In [10]:
# Accuracy of the model trained on all data 
compute_all_accuracy(matrix_data,charac_list,575)

0.896551724137931

In [11]:
# Accuracy of training data with model trained on training data 
def compute_train_accuracy(data,subtype,seed,split): 
    
    #Split the data into 50% train and 50% test 
    
    X_train, X_test, y_train, y_test = train_test_split(data,subtype,stratify = charac_list,test_size=split,random_state = seed)
    
    #develope the kmeans model
    kmeans = KMeans(n_clusters = 2, random_state = seed).fit(X_train)
    
    # Predict label(in the form of array)
    predict_label = kmeans.labels_
    # convert predict label array into list 
    predict_label = predict_label.tolist()
    
    # Create a dataframe for predict label and subtype
    df = pd.DataFrame([predict_label,y_train]).T
    df.columns = ["Cluster", "Subtype"]
    
    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]
    
    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]
    
    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}
    
    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in y_train]
    # Compute accuracy_score 
    return accuracy_score(train_label, predict_label)

In [12]:
# Accuracy of the model trained on 50% of training data 
compute_train_accuracy(matrix_data,charac_list,575,0.5)

0.9655172413793104

In [13]:
# Accuracy of testing data with model trained on training data 
def compute_test_accuracy(data,subtype,seed,split):
    
    # Split all data into 50% train and 50% test
    X_train, X_test, y_train, y_test = train_test_split(data,subtype,stratify = charac_list,test_size=split,random_state = seed)
    
    # Develop kmeans model trained based on training data 
    kmeans = KMeans(n_clusters = 2, random_state = 575).fit(X_train)
    
    # Predict label for testing data
    predict_label = kmeans.fit_predict(X_test)

    # Predict label for testing data
    predict_label = kmeans.fit_predict(X_test)
    # convert predict label array into list 
    predict_label = predict_label.tolist()

    # Create a dataframe for predict label and subtype(test label)
    df = pd.DataFrame([predict_label,y_test]).T
    df.columns = ["Cluster", "Subtype"]

    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]

    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]

    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}

    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in y_test]
    
    return accuracy_score(train_label,predict_label)

In [14]:
# Accuracy of the model trained on 50% of testing data 
compute_test_accuracy(matrix_data,charac_list,575,0.5)

0.8275862068965517